In [ ]:
!sudo apt-get install openjdk-8-jdk
!java -version
!pip install --ignore-installed -q pyspark==2.4.4
!pip install spark-nlp
!pip install streamlit
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

In [ ]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
spark = sparknlp.start()

In [ ]:
### Select the model and re-run all the cells below ####
MODEL_NAME='wikiner_840B_300'
model_dict={'wikiner_840B_300': 'glove_840B_300',
            'wikiner_6B_100' : 'glove_100d',
            'wikiner_6B_300': 'glove_6B_300'
}

In [ ]:
## Generating Example Files ##
text_list = ["""William Henry Gates III (ur. 28 października 1955 r.) To amerykański magnat biznesowy, programista, inwestor i filantrop. Najbardziej znany jest jako współzałożyciel Microsoft Corporation. Podczas swojej kariery w Microsoft Gates zajmował stanowiska prezesa, dyrektora generalnego (CEO), prezesa i głównego architekta oprogramowania, będąc jednocześnie największym indywidualnym akcjonariuszem do maja 2014 r. Jest jednym z najbardziej znanych przedsiębiorców i pionierów rewolucja mikrokomputerowa lat 70. i 80. Urodzony i wychowany w Seattle w stanie Waszyngton, Gates był współzałożycielem Microsoftu z przyjacielem z dzieciństwa Paulem Allenem w 1975 r. W Albuquerque w Nowym Meksyku; stała się największą na świecie firmą produkującą oprogramowanie komputerowe. Gates prowadził firmę jako prezes i dyrektor generalny, aż do ustąpienia ze stanowiska dyrektora generalnego w styczniu 2000 r., Ale pozostał przewodniczącym i został głównym architektem oprogramowania. Pod koniec lat 90. Gates był krytykowany za taktykę biznesową, którą uważano za antykonkurencyjną. Opinię tę podtrzymują liczne orzeczenia sądowe. W czerwcu 2006 r. Gates ogłosił, że przejdzie do pracy w niepełnym wymiarze godzin w Microsoft i pracy w pełnym wymiarze godzin w Bill & Melinda Gates Foundation, prywatnej fundacji charytatywnej, którą on i jego żona Melinda Gates utworzyli w 2000 r. [ 9] Stopniowo przeniósł obowiązki na Raya Ozziego i Craiga Mundie. Zrezygnował z funkcji prezesa Microsoftu w lutym 2014 r. I objął nowe stanowisko jako doradca ds. Technologii, aby wesprzeć nowo mianowaną CEO Satyę Nadellę.""",
 """Mona Lisa to XVI-wieczny obraz olejny stworzony przez Leonarda. Odbywa się w Luwrze w Paryżu.""" 
]


In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# italian ner is traing with glove_300d. So we use the same embeddings in the pipeline
lang = 'xx'
if model_dict[MODEL_NAME] == 'glove_100d':
  lang='en'
embeddings = WordEmbeddingsModel.pretrained(model_dict[MODEL_NAME], lang=lang).\
                    setInputCols(["document", 'token']).\
                    setOutputCol("embeddings")

public_ner = NerDLModel.pretrained(MODEL_NAME, 'es') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

ner_converter = NerConverter() \
                .setInputCols(["document", "token", "ner"]) \
                  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[ documentAssembler, 
                                 tokenizer,
                                 embeddings,
                                 public_ner,
                                 ner_converter
                                 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

glove_840B_300 download started this may take some time.
Approximate size to download 2.3 GB
[OK!]
wikiner_840B_300 download started this may take some time.
Approximate size to download 14.2 MB
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)
#result = result.toPandas()

+----------------------------------------------------------------------------+---------+
|chunk                                                                       |ner_label|
+----------------------------------------------------------------------------+---------+
|William Henry Gates III                                                     |PER      |
|To amerykański magnat biznesowy                                             |MISC     |
|Microsoft Corporation                                                       |ORG      |
|Podczas swojej kariery w Microsoft Gates                                    |MISC     |
|Jest jednym                                                                 |MISC     |
|Urodzony i wychowany w Seattle w stanie Waszyngton                          |MISC     |
|Gates                                                                       |PER      |
|Microsoftu z przyjacielem                                                   |MISC     |
|Paulem Allenem w 197